# Quantum Integer Programming (QuIP) 47-779. Fall 2020, CMU

This notebook contains material from the Quantum Integer Programming Lecture at CMU Fall 2020 by David Bernal (bernalde at cmu.edu), Sridhar Tayur (stayur at cmu.edu) and Davide Venturelli; the content is available on **[Github](https://github.com/bernalde/QuIP)**. The text is released under the **[CC-BY-NC-ND-4.0](https://creativecommons.org/licenses/by-nc-nd/4.0/legalcode) license, and code is released under the **[MIT license](https://opensource.org/licenses/MIT).*

This notebook makes simple computations of Graver basis. Because of the complexity of these computation, we suggest that for more complicated problems you install the excellent **[4ti2](https://4ti2.github.io/)** software, an open-source implementation of several routines useful for the study of integer programming through algebraic geometry. It can be used as a stand-alone library and call it from C++ or from Python. In python, there are two ways of accessing it, either through **[Sage](https://www.sagemath.org/)** (which is an open-source mathematics software) or directly compiling it and installing a thing **[Python wrapper](https://github.com/alfsan/Py4ti2)**

Run in **[Google Colab](https://colab.research.google.com/github/bernalde/QuIP/blob/master/notebooks/Notebook%203%20-%20Graver%20basis.ipynb)**

## Introduction to Graver basis computation
### To be modified

In [1]:
# If using this on Google collab, if not we can import 4ti2
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

# Let's start with Pyomo
if not IN_COLAB:
    from Py4ti2int32 import *

In [2]:
from sympy import *
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from typing import Callable
import itertools

In [3]:
# Fist example
A = np.array([[1, 1, 1, 1], [1, 5, 10, 25]])
b = np.array([[21],[156]])

# Second example
# A = np.array([[1, 1, 1, 1], [1, 2, 3, 4]])
# b = np.array([[10], [21]])
# c = np.array([0, 1, 0, 2])

# Third example
# A = np.array([[1, 1, 1, 1], [0, 1, 2, 3]])
# b = np.array([[10],[15]])
# c = np.array([1, 3, 14, 17])

# Fourth example
# A = np.array([[1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,0,1,1,1,0,1,0,1,0],
# [1,1,1,1,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,1,1,1,1,1,1],
# [0,1,0,0,0,1,0,1,0,1,1,0,1,1,0,1,1,0,0,1,0,0,1,1,1],
# [0,0,0,0,0,0,0,1,0,1,1,1,0,1,1,1,1,0,0,1,0,0,0,0,0],
# [0,1,1,1,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0]])
# b = np.array([[9], [8], [7], [5], [5]])
if not IN_COLAB:
    r = graver("mat", A.tolist())
else:
    r = [[5, -6, 0, 1], [5, -9, 4, 0], [0, 3, -4, 1], [5, -3, -4, 2], [5, 0, -8, 3]]
# print(r)
r = np.array(r)

In [4]:
x0 = np.array([1,15,3,2])
# x0 = np.array([1,8,0,1])
# x0 = np.array([3,0,6,1])
# x0 = np.array([1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, -2,
#                1, 0, -1, 0, 1, -1, 1, -2, -2, 1, 1, 1])

x_lo = -2*np.ones_like(x0)
x_up = 2*np.ones_like(x0)
print(np.dot(A,x0))
print(b.transpose())
print(np.array_equiv(np.dot(A, x0), b.T))


[ 21 156]
[[ 21 156]]
True


In [5]:
# Objective function definition

epsilon = 0.01
mu = np.random.rand(len(x0))
sigma = np.multiply(np.random.rand(len(x0)),mu)
print(mu)
print(sigma)
def f(x):
    return np.sum(np.abs(x - 5))
    # return np.dot(c,x)
    # return -np.dot(mu, x) + np.sqrt(((1-epsilon)/epsilon)*np.dot(sigma**2, x**2))

def const(x):
    return np.array_equiv(np.dot(A,x),b.T) or np.array_equiv(np.dot(A,x),b)


[0.54713288 0.33800584 0.0667273  0.30760454]
[0.35235475 0.13878311 0.04155517 0.07482565]


In [6]:
# Define rules to choose augmentation element, either the best one (argmin) or the first one that is found
def argmin(iterable):
    return min(enumerate(iterable), key=lambda x: x[1])

def greedy(iterable):
    for i, val in enumerate(iterable):
        if val[1] != 0:
            return i, val
    else:
        return i, val

In [7]:
# Bisection rules for finding best step size
def bisection(g: np.ndarray, fun: Callable, x: np.ndarray, x_lo: np.ndarray = None, x_up: np.ndarray = None, laststep: np.ndarray = None) -> (float, int):
    if np.array_equal(g, laststep):
        return (fun(x), 0)
    if x_lo is None:
        x_lo = np.zeros_like(x)
    if x_up is None:
        x_up = np.ones_like(x)*max(x)*2

    u = max(x_up) - min(x_lo)
    l = -(max(x_up) - min(x_lo))
    for i, gi in enumerate(g):
        if gi >= 1:
            if np.floor((x_up[i] - x[i]) / gi) < u:
                u = int(np.floor((x_up[i] - x[i]) / gi))
            if np.ceil((x_lo[i] - x[i]) / gi) > l:
                l = int(np.ceil((x_lo[i] - x[i]) / gi))
        elif gi <= -1:
            if np.ceil((x_up[i] - x[i]) / gi) > l:
                l = int(np.ceil((x_up[i] - x[i]) / gi))
            if np.floor((x_lo[i] - x[i]) / gi) < u:
                u = int(np.floor((x_lo[i] - x[i]) / gi))
    alpha = u

    while u - l > 1:
        if fun(x + l*g) < fun(x + u*g):
            alpha = l
        else:
            alpha = u
        p1 = int(np.floor((l+u)/2) - 1)
        p2 = int(np.floor((l+u)/2))
        p3 = int(np.floor((l+u)/2) + 1)
        if fun(x + p1*g) < fun(x + p2*g):
            u = int(np.floor((l+u)/2))
        elif fun(x + p3*g) < fun(x + p2*g):
            l = int(np.floor((l+u)/2) + 1)
        else:
            alpha = p2
            break

    if fun(x + l*g) < fun(x + u*g) and fun(x + l*g) < fun(x + alpha*g):
        alpha = l
    elif fun(x + u*g) < fun(x + alpha*g):
        alpha = u

    return (fun(x + alpha*g), alpha)

In [8]:
# We can just have a single step move (works well with greedy approach)
def single_move(g: np.ndarray, fun: Callable, x: np.ndarray, x_lo: np.ndarray = None, x_up: np.ndarray = None, laststep: np.ndarray = None) -> (float, int):
    if x_lo is None:
        x_lo = np.zeros_like(x)
    if x_up is None:
        x_up = np.ones_like(x)*max(x)*2

    alpha = 0

    if (x + g <= x_up).all() and (x + g >= x_lo).all():
        if fun(x + g) < fun(x):
            alpha = 1
    elif (x - g <= x_up).all() and (x - g >= x_lo).all():
        if fun(x - g) < fun(x) and fun(x - g) < fun(x + g):
            alpha = -1
    
    return (fun(x + alpha*g), alpha)

In [9]:
# Let's perform the augmentation
dist = 1
gprev = None
while dist != 0:
    g1, (obj, dist) = argmin(
        bisection(e, f, x0, laststep=gprev, x_lo=x_lo, x_up=x_up) for e in r)

    # g1, (obj, dist) = greedy(
    #     bisection(e, f, x0, laststep=gprev, x_lo=x_lo, x_up=x_up) for e in r)
    # g1, (obj, dist) = greedy(
    #     single_move(e, f, x0, x_lo=x_lo, x_up=x_up) for e in r)
    print('iteration')
    print(x0)
    x0 = x0 + r[g1]*dist
    gprev = r[g1]
    print(g1, (obj, dist))
    print(x0)
    print(const(x0))
    print(dist)

iteration
[ 1 15  3  2]
0 (19, 0)
[ 1 15  3  2]
True
0
